In [1]:
from pyspark.sql import SQLContext

sc
sqlContext = SQLContext(sc)

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1617376874076_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
companies = sqlContext.read.option("header", True).csv('s3://cse6242-scasalis3/s_and_p_500_list.csv')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
companies.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+--------------------+--------------------+
|Symbol|                Name|              Sector|
+------+--------------------+--------------------+
|   MMM|          3M Company|         Industrials|
|   AOS|     A.O. Smith Corp|         Industrials|
|   ABT| Abbott Laboratories|         Health Care|
|  ABBV|         AbbVie Inc.|         Health Care|
|  ABMD|             Abiomed|         Health Care|
|   ACN|           Accenture|Information Techn...|
|  ATVI| Activision Blizzard|Communication Ser...|
|  ADBE|          Adobe Inc.|Information Techn...|
|   AAP|  Advance Auto Parts|Consumer Discreti...|
|   AMD|Advanced Micro De...|Information Techn...|
|   AES|            AES Corp|           Utilities|
|   AFL|               Aflac|          Financials|
|     A|Agilent Technologies|         Health Care|
|   APD|Air Products & Ch...|           Materials|
|  AKAM| Akamai Technologies|Information Techn...|
|   ALK|    Alaska Air Group|         Industrials|
|   ALB|Albemarle Corpora...|  

In [35]:
from pyspark.sql.functions import col

company_symbols = set(map(lambda x: x[0], \
                          companies.withColumn("Symbol", col("Symbol")).select("Symbol").collect()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
from pyspark.sql.functions import *
import itertools

def filter_tweets(year_partition, month_partition, day_partition, hour_partition):
    tweets = sqlContext.read.json('s3://cse6242-scasalis3/twitter/' + year_partition + '/' + month_partition + '/' + day_partition + '/' + hour_partition + '/', \
                                  samplingRatio = 0.01)
    filtered = tweets.filter(col('lang') == 'en') \
                     .withColumn('parsed_created_at', from_unixtime(unix_timestamp('created_at', 'EEE MMM d HH:mm:ss z yyyy'))) \
                     .withColumn('year', year(col('parsed_created_at'))) \
                     .withColumn('month', month(col('parsed_created_at'))) \
                     .withColumn('day', dayofmonth(col('parsed_created_at'))) \
                     .withColumn('hour', hour(col('parsed_created_at'))) \
                     .filter(col('parsed_created_at').isNotNull()) \
                     .withColumn('Symbol', regexp_extract("text", '(^|\s)(\b' + '|'.join(map(lambda x: x[0] + x[1], itertools.product(['$', '#'], company_symbols))) + '\b)(\s|$)', 2)) \
                     .fillna('', ['Symbol']) \
                     .filter(col('Symbol') != '') \
                     .withColumn('Symbol', expr('substring(Symbol, 2, length(Symbol))')) \
                     .join(companies, 'Symbol', 'left_outer') \
                     .withColumnRenamed('id_str', 'tweet_id') \
                     .withColumnRenamed('Name', 'company_name') \
                     .select('Symbol', 'company_name', 'Sector', 'year', 'month', 'day', 'hour', 'text', 'user.name', \
                             'user.screen_name', 'user.verified', 'user.followers_count', 'user.friends_count', \
                             'user.favourites_count', 'user.statuses_count', 'quote_count', 'reply_count', \
                             'retweet_count', 'favorite_count', 'tweet_id', 'user.id_str')
    return (tweets, filtered)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
(all_tweets, filtered_tweets) = filter_tweets('*', '*', '*', '*')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
filtered_tweets.write.partitionBy('Symbol').csv('s3://cse6242-scasalis3/filtered-twitter/')

In [38]:
all_tweets.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

918297066

In [ ]:
filtered_tweets.count()

In [8]:
from pyspark.sql.functions import col

stock_tags = ["stockmarket", "stocks", "stock", "trading", "market", "investing", "alpha", "seekingalpha", \
              "analysis", "finance", "time", "daytrading", "business", "inc", "alphanews", "seeking", \
              "source", "equity"]

company_symbol_tweets = tweets.filter(col('lang') == 'en') \
                              .filter(col("text").rlike('(^|\s)(\b' + '|'.join(map(lambda x: "#" + x,stock_tags)) + '\b)(\s|$)')) \
                              .filter(col("text").rlike('(^|\s)(\b' + '|'.join(company_symbols) + '\b)(\s|$)')) \
                              .select("text")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
company_symbol_tweets.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

19

In [38]:
company_symbol_tweets.limit(100).collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(text='$BBIG Announces Closing Of A Merger Agreement. https://t.co/H51dIdaMv8 #wsj #nytimes #business #reuters… https://t.co/GHdQaaQe09'), Row(text='$BSFFF - Basic-Fit: A Risky Bet On An Unexpected Beneficiary From COVID-19. https://t.co/0blFWrts8I #investing #markets #finance'), Row(text='$ALT - Altimmune: A COVID-19 Winner After All. https://t.co/dhzjZn5xd1 #finance #business #stockmarket'), Row(text='RT @sallyeaves: A #FutureReady Deep Dive - #data visualisation to improve #business readiness for #change enabling personalisation &amp; #bigda…')]

In [36]:
companies.filter(col("Symbol") == "PM").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+--------------------+----------------+
|Symbol|                Name|          Sector|
+------+--------------------+----------------+
|    PM|Philip Morris Int...|Consumer Staples|
+------+--------------------+----------------+

In [5]:
tweets.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

5315307

In [10]:
from pyspark.sql.functions import col
import itertools

company_symbol_tweets = tweets.filter(col('lang') == 'en') \
                              .filter(col("text").rlike('(^|\s)(\b' + '|'.join(map(lambda x: x[0] + x[1], itertools.product(['$', '#'], company_symbols))) + '\b)(\s|$)')) \
                              .select("text")

company_symbol_tweets.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1624

In [68]:
filtered_tweets = filter_tweets('2021', '01', '01', '01')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

218617

In [69]:
filtered_tweets.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+------------+--------------------+----+-----+---+----+--------------------+--------------------+---------------+--------+---------------+-------------+----------------+--------------+-----------+-----------+-------------+--------------+----------------------+
|Symbol|        Name|              Sector|year|month|day|hour|                text|                name|    screen_name|verified|followers_count|friends_count|favourites_count|statuses_count|quote_count|reply_count|retweet_count|favorite_count|              hashtags|
+------+------------+--------------------+----+-----+---+----+--------------------+--------------------+---------------+--------+---------------+-------------+----------------+--------------+-----------+-----------+-------------+--------------+----------------------+
|     K| Kellogg Co.|    Consumer Staples|2021|    1|  1|   8|RT @BigHitEnt_JP:...|             jara ᵕ̈|   jaejaejehyun|   false|           1316|         1310|            8164|         21650|     

In [27]:
filtered_tweets.groupBy('Symbol').count().show(1000)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+-------+
|Symbol|  count|
+------+-------+
|  INFO|      4|
|   NSC|     10|
|   FOX|      4|
|   CAG|      4|
|   ADP|      1|
|   CMS|      2|
|   GIS|      3|
|    MS|      5|
|   TSN|      2|
|    SO|      2|
|   NEM|      2|
|    MO|      1|
|     D|      6|
|    BR|      1|
|   ESS|      2|
|   ALL|      1|
|    DE|      1|
|  DISH|      1|
|  ULTA|      1|
|   PPG|      1|
|  GILD|      4|
|   WBA|     13|
|   MCD|      2|
|   ABC|      9|
|    FE|      2|
|    ED|      3|
|    RL|      2|
|   SNA|      3|
|    RF|      1|
|   HST|      1|
|   FTV|      1|
|   LEN|      1|
|    EW|      1|
|    GE|      2|
|   UPS|      1|
|   SLB|      1|
|    CL|     14|
|     L|      3|
|   MLM|      2|
|  ETSY|      2|
|   IBM|      4|
|    NI|      1|
|   IFF|      1|
|   GPN|      4|
|    GM|      4|
|   TFC|      2|
|   LMT|      2|
|    IR|      2|
|      |7205287|
|     T|      4|
|   WEC|      8|
|   APH|      2|
|   UDR|      1|
|   MCO|      1|
|    KO|      2|
|  CERN|      

In [24]:
tweets = sqlContext.read.json('s3://cse6242-scasalis3/twitter/2020/12/01/01/', samplingRatio = 0.01)
tweets.schema

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

StructType(List(StructField(contributors,StringType,true),StructField(coordinates,StringType,true),StructField(created_at,StringType,true),StructField(delete,StructType(List(StructField(status,StructType(List(StructField(id,LongType,true),StructField(id_str,StringType,true),StructField(user_id,LongType,true),StructField(user_id_str,StringType,true))),true),StructField(timestamp_ms,StringType,true))),true),StructField(display_text_range,ArrayType(LongType,true),true),StructField(entities,StructType(List(StructField(hashtags,ArrayType(StructType(List(StructField(indices,ArrayType(LongType,true),true),StructField(text,StringType,true))),true),true),StructField(media,ArrayType(StructType(List(StructField(additional_media_info,StructType(List(StructField(description,StringType,true),StructField(embeddable,BooleanType,true),StructField(monetizable,BooleanType,true),StructField(title,StringType,true))),true),StructField(description,StringType,true),StructField(display_url,StringType,true),Str

In [64]:
tweets.filter(col('lang') == 'en') \
                 .withColumn('parsed_created_at', from_unixtime(unix_timestamp('created_at', 'EEE MMM d HH:mm:ss z yyyy'))) \
                 .withColumn('year', year(col('parsed_created_at'))) \
                 .withColumn('month', month(col('parsed_created_at'))) \
                 .withColumn('day', dayofmonth(col('parsed_created_at'))) \
                 .withColumn('hour', hour(col('parsed_created_at'))) \
                 .filter(col('parsed_created_at').isNotNull()) \
                 .withColumn('Symbol', regexp_extract("text", '(^|\s)(\b' + '|'.join(map(lambda x: x[0] + x[1], itertools.product(['$', '#'], company_symbols))) + '\b)(\s|$)', 2)) \
                 .fillna('', ['Symbol']) \
                 .filter(col('Symbol') != '') \
                 .withColumn('Symbol', expr('substring(Symbol, 2, length(Symbol))')) \
                 .join(companies, 'Symbol', 'left_outer') \
                 .select('Symbol', 'Name', 'Sector', 'year', 'month', 'day', 'hour', 'text', 'user.name', \
                         'user.screen_name', 'user.verified', 'user.followers_count', 'user.friends_count', \
                         'user.favourites_count', 'user.statuses_count', 'quote_count', 'reply_count', \
                         'retweet_count', 'favorite_count', 'entities.hashtags').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+--------------------+--------------------+----+-----+---+----+--------------------+--------------------+---------------+--------+---------------+-------------+----------------+--------------+-----------+-----------+-------------+--------------+----------------------+
|Symbol|                Name|              Sector|year|month|day|hour|                text|                name|    screen_name|verified|followers_count|friends_count|favourites_count|statuses_count|quote_count|reply_count|retweet_count|favorite_count|              hashtags|
+------+--------------------+--------------------+----+-----+---+----+--------------------+--------------------+---------------+--------+---------------+-------------+----------------+--------------+-----------+-----------+-------------+--------------+----------------------+
|   TXT|        Textron Inc.|         Industrials|2020|   12|  1|   8|RT @TXTBigHitWW: ...|       GYUmmy🐻𖧵💜✜|         Bwi4me|   false|            154|          219|        

In [6]:
from pyspark.sql.types import *

schema = StructType([StructField("Symbol", StringType(), True), \
                     StructField("company_name", StringType(), True), \
                     StructField("Sector", StringType(), True), \
                     StructField("year", StringType(), True), \
                     StructField("month", StringType(), True), \
                     StructField("day", StringType(), True), \
                     StructField("hour", StringType(), True), \
                     StructField("text", StringType(), True), \
                     StructField("user_name", StringType(), True), \
                     StructField("user_screen_name", StringType(), True), \
                     StructField("user_verified", BooleanType(), True), \
                     StructField("user_followers_count", IntegerType(), True), \
                     StructField("user_friends_count", IntegerType(), True), \
                     StructField("user_favourites_count", IntegerType(), True), \
                     StructField("user_statuses_count", IntegerType(), True), \
                     StructField("quote_count", IntegerType(), True), \
                     StructField("reply_count", IntegerType(), True), \
                     StructField("retweet_count", IntegerType(), True), \
                     StructField("favorite_count", IntegerType(), True), \
                     StructField("tweet_id", StringType(), True), \
                     StructField("user_id", StringType(), True) \
                     ])

filtered_tweets = sqlContext.read.csv('s3://cse6242-scasalis3/filtered-twitter/', schema)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
filtered_tweets.repartition('Symbol') \
               .write \
               .option('header', True) \
               .partitionBy('Symbol') \
               .csv('s3://cse6242-scasalis3/filtered-twitter-repartition3/')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
from pyspark.sql.functions import col

filtered_tweets.groupBy('Symbol').count().orderBy(col('count').desc()).coalesce(1) \
               .write.option('header', True).csv('s3://cse6242-scasalis3/filtered-twitter-stats2/')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
filtered_tweets.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

220074